Verinin anlatılması, açıklanması.

Kullandığım veri seti Online Payments Fraud Detection Dataseti. Veri seti yapılan para işlemlerinden hangisinin dolandırıcılık işlemi olup olmadığını gösteren bir veri seti.  

Veri setindeki özellikler:

    1- step: işlem süresini saat cinsinden ifade eder.
    2- type: online işlemin tipi.
    3- amount: işlemin miktarı.
    4- nameOrig: işlemi başlatan müşterinin adı.
    5- oldbalanceOrg: işlem öncesi hesap bakiyesi.
    6- newbalanceOrig: işlem sonrası hesap bakiyesi.
    7- nameDest: işlemi alan hesabın adı.
    8- oldbalanceDest: işlemin ulaştığı hesabın işlem öncesi bakiyesi.
    9- newbalanceDest: işlemin ulaştığı hesabın işlem sonrası bakiyesi.
    10- isFraud: dolandırıcılık işlemi olup olmadığı.

Bu veri setindeki asıl amaç 1 den 9 a kadar verilen özellikler ile yapılan işlemin dolandırıcılık işlemi olup olmadığının tahminlenmesi.

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#EDA - Keşifsel Veri Analizi
import pandas as pd

#datanın pandas kütüphanesi yardımıyla okunması
data = pd.read_csv("/home/tugberk/Downloads/archive/PS_20174392719_1491204439457_log.csv")

In [5]:
#Verinin ilk 5 satırının incelenmesi
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
#Verinin son 5 satırının incelenmesi
data.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0
6362619,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [8]:
#Veri setimizdeki değerlerin kaç tane farklı değere sahip olduğunu görmek...
data.nunique()

step                  743
type                    5
amount            5316900
nameOrig          6353307
oldbalanceOrg     1845844
newbalanceOrig    2682586
nameDest          2722362
oldbalanceDest    3614697
newbalanceDest    3555499
isFraud                 2
isFlaggedFraud          2
dtype: int64

In [9]:
#Verimizde null değer var mı görmek için...
data.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [10]:
#Veri setimizden step, nameOrig ve nameDest sutünlarını 
#siliyoruz çünkü tahminleme algortimasında herhangi bir işimize yaramayacaklar
data = data.drop(['step', 'nameOrig', 'nameDest'], axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   type            object 
 1   amount          float64
 2   oldbalanceOrg   float64
 3   newbalanceOrig  float64
 4   oldbalanceDest  float64
 5   newbalanceDest  float64
 6   isFraud         int64  
 7   isFlaggedFraud  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 388.3+ MB


In [11]:
#Bu adımda veri setindeki özelliklerin aritmetik ortalaması, std ve veri setindeki max min değerlerini görebiliyoruz.
data.describe().T

,count,mean,std,min,25%,50%,75%,max
amount,6362620.0,1.798619e+05,6.038582e+05,0.0,13389.57,74871.940,2.087215e+05,9.244552e+07
oldbalanceOrg,6362620.0,8.338831e+05,2.888243e+06,0.0,0.00,14208.000,1.073152e+05,5.958504e+07
newbalanceOrig,6362620.0,8.551137e+05,2.924049e+06,0.0,0.00,0.000,1.442584e+05,4.958504e+07
oldbalanceDest,6362620.0,1.100702e+06,3.399180e+06,0.0,0.00,132705.665,9.430367e+05,3.560159e+08
newbalanceDest,6362620.0,1.224996e+06,3.674129e+06,0.0,0.00,214661.440,1.111909e+06,3.561793e+08
isFraud,6362620.0,1.290820e-03,3.590480e-02,0.0,0.00,0.000,0.000000e+00,1.000000e+00
isFlaggedFraud,6362620.0,2.514687e-06,1.585775e-03,0.0,0.00,0.000,0.000000e+00,1.000000e+00


In [12]:
#Veri setimizdeki object ve numerik değerler olan sutünları ayırıyouz ki sunması kolay olsun
cat_cols=data.select_dtypes(include=['object']).columns
num_cols = data.select_dtypes(include=np.number).columns.tolist()
print("Categorical Variables:")
print(cat_cols)
print("Numerical Variables:")
print(num_cols)

Categorical Variables:
Index(['type'], dtype='object')
Numerical Variables:
['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']


In [13]:
#scikit learn kütühanesi yardımı ile verimizdeki object veri tipi olan sutünlarımızı numerik değerlere çeviriyoruz.
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

data["type"] = label_encoder.fit_transform(data["type"])

In [17]:
data.head() 

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,3,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,3,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,4,181.00,181.0,0.00,0.0,0.0,1,0
3,1,181.00,181.0,0.00,21182.0,0.0,1,0
4,3,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [19]:
#Algoritma seçimi ve Hiperparametre Optimizasyonu
#Bu kısımda lineer regresyon ile knn sınıflandırma algoritmalarını kullanacağım

#verimizi x ye y olacak şekilde ikiye ayıralım
#x sutünları verimizi tahminlemek için kullanacağımız özellikler olacak
#y sutünu verimizin tahminlemeye çalışıtığımız veri olacak.

x = data.iloc[:, 0:-2]

y = data["isFraud"]

MemoryError: Unable to allocate 295. TiB for an array with shape (6362620, 6362620) and data type float64

Hangi algoritma neden seçildi, seçilen 2 algoritmadan hangisi nasıl sonuç çıkardı neden iyi/kötü sonuç çıkardı.

In [16]:
#Model Evaluation

Model üzerinde değerlendirme algoritmaları kullanarak sonuçların karşılaştırılması.